In [5]:
import os
import shutil

# 이미지와 라벨 경로
image_root = "/home/hyun/dev_ws/ROS_project/training/fire/data/image"
label_root = "/home/hyun/dev_ws/ROS_project/training/fire/data/label"

# 평탄화된 디렉토리 경로
flat_image_dir = "/home/hyun/dev_ws/ROS_project/training/fire/data/flat_image"
flat_label_dir = "/home/hyun/dev_ws/ROS_project/training/fire/data/flat_label"

os.makedirs(flat_image_dir, exist_ok=True)
os.makedirs(flat_label_dir, exist_ok=True)

# 이미지 평탄화
for root, _, files in os.walk(image_root):
    for file in files:
        if file.endswith(('.jpg', '.png')):
            src = os.path.join(root, file)
            dst = os.path.join(flat_image_dir, file)
            shutil.copy(src, dst)

# 라벨 평탄화
for root, _, files in os.walk(label_root):
    for file in files:
        if file.endswith('.txt'):
            src = os.path.join(root, file)
            dst = os.path.join(flat_label_dir, file)
            shutil.copy(src, dst)

print("이미지와 라벨이 평탄화되었습니다.")


이미지와 라벨이 평탄화되었습니다.


In [3]:
from glob import glob
img_list = glob('/home/hyun/dev_ws/ROS_project/training/fire/data/mydata/train/images/*.jpg')
label_list = glob('/home/hyun/dev_ws/ROS_project/training/fire/data/mydata/train/labels/*.txt')
print(len(img_list), len(label_list))

15392 15392


In [4]:
from sklearn.model_selection import train_test_split

# 8:1:1 비율로 데이터 분할
train_img_list, temp_img_list = train_test_split(img_list, test_size=0.2, random_state=13)
val_img_list, test_img_list = train_test_split(temp_img_list, test_size=0.5, random_state=13)

# 데이터 개수 출력
print(len(train_img_list), len(val_img_list), len(test_img_list))


12313 1539 1540


In [5]:
import os

# 리스트를 텍스트 파일로 저장
with open("train.txt", "w") as f:
    f.write("\n".join(train_img_list))

with open("val.txt", "w") as f:
    f.write("\n".join(val_img_list))

with open("test.txt", "w") as f:
    f.write("\n".join(test_img_list))


In [6]:
from ultralytics import YOLO

# YOLOv8s 모델 로드
model = YOLO("yolov8s.pt")  # 사전 학습된 YOLOv8s 모델 사용

# 학습 수행
model.train(
    data="/home/hyun/dev_ws/ROS_project/training/fire/data.yaml",  # 데이터셋 설정 파일
    epochs=100,  # 최대 100 에포크 학습
    batch=16,  # 배치 크기
    imgsz=640,  # 이미지 크기
    device="cuda",  # GPU 사용 (없으면 "cpu")
    patience=10,  # 10 에포크 동안 개선이 없으면 Early Stopping
    save=True,  # 모델 저장
    save_period=5,  # 5 에포크마다 모델 저장 (선택 사항)
)

# 베스트 모델 저장 경로 출력
print(f"Best model saved at: {model.best}")



100%|██████████| 21.5M/21.5M [00:00<00:00, 64.1MB/s]


New https://pypi.org/project/ultralytics/8.3.74 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.65 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/hyun/dev_ws/ROS_project/training/fire/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

100%|██████████| 5.35M/5.35M [00:00<00:00, 61.7MB/s]


AMP: checks passed ✅


train: Scanning /home/hyun/dev_ws/ROS_project/training/fire/data/mydata/train/labels... 12313 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12313/12313 [00:04<00:00, 2698.21it/s]


...

                   all       1539       1539          1          1      0.995      0.944
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 59, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



69 epochs completed in 3.983 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.65 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:07<00:00,  6.29it/s]


                   all       1539       1539          1          1      0.995      0.945
                  fire        519        519          1          1      0.995      0.897
                 smoke       1020       1020          1          1      0.995      0.992
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train


AttributeError: 'DetectionModel' object has no attribute 'best'

In [3]:
from ultralytics import YOLO

# 학습된 모델 로드
model = YOLO('//home/hyun/dev_ws/ROS_project/model_store/fire_detector/fire.pt')  # 학습된 가중치 경로

# 추론할 이미지 경로
image_path = '/home/hyun/dev_ws/ROS_project/fire.jpg'

# 이미지 추론
results = model(image_path)

# 결과를 저장할 디렉토리 설정
output_dir = '/home/hyun/dev_ws/ROS_project/data/output'

# 디렉토리가 없으면 생성
import os
os.makedirs(output_dir, exist_ok=True)

# 각 결과를 저장
for idx, result in enumerate(results):
    # 저장할 파일명 설정
    save_path = os.path.join(output_dir, f"result_{idx}.jpg")
    result.plot(save=True, filename=save_path)  # 결과를 저장

# 저장된 결과 확인
print(f"Results saved to {output_dir}")



image 1/1 /home/hyun/dev_ws/ROS_project/fire.jpg: 448x640 1 smoke, 170.1ms
Speed: 3.6ms preprocess, 170.1ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /home/hyun/dev_ws/ROS_project/data/output


In [6]:
from ultralytics import YOLO
import cv2
import os

# 학습된 YOLO 모델 로드
model = YOLO('/home/hyun/dev_ws/ROS_project/training/fire/runs/detect/train/weights/epoch10.pt')  # 학습된 가중치 경로

# OpenCV로 카메라 열기
video_path = "/home/hyun/dev_ws/ROS_project/[속보] 인천 구월동 로데오거리 상가 건물에서 화재 [NRODWJVwRT0].mp4"
cap = cv2.VideoCapture(0)  # 0은 기본 카메라, 다른 카메라를 사용하려면 번호 변경

# 결과 저장 디렉토리 설정
output_dir = '/home/hyun/dev_ws/ROS_project/data/output'
os.makedirs(output_dir, exist_ok=True)

# 실시간 영상 처리
while cap.isOpened():
    ret, frame = cap.read()  # 카메라에서 프레임 읽기
    if not ret:
        print("카메라에서 프레임을 읽을 수 없습니다.")
        break

    # YOLO 모델로 추론
    results = model(frame)

    # 결과를 이미지에 그리기
    annotated_frame = results[0].plot()

    # 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 결과를 파일로 저장 (필요한 경우)
    save_path = os.path.join(output_dir, "realtime_result.jpg")
    cv2.imwrite(save_path, annotated_frame)

# 자원 정리
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 smoke, 172.7ms
Speed: 1.3ms preprocess, 172.7ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 smoke, 165.2ms
Speed: 1.0ms preprocess, 165.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 smoke, 179.2ms
Speed: 2.0ms preprocess, 179.2ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 178.8ms
Speed: 1.3ms preprocess, 178.8ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 smoke, 185.1ms
Speed: 1.5ms preprocess, 185.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 185.3ms
Speed: 1.7ms preprocess, 185.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 smoke, 184.5ms
Speed: 1.6ms preprocess, 184.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 179.2ms
Speed: 1.5ms preprocess, 179.2ms inference, 0.5ms postproc

In [8]:
from ultralytics import YOLO

# ✅ 학습된 모델 로드
model = YOLO("/home/hyun/dev_ws/ROS_project/model_store/fire_detector/fire.pt")

# ✅ 테스트 데이터 평가 수행
metrics = model.val(
    data="/home/hyun/dev_ws/ROS_project/training/fire/data.yaml",  # ✅ data.yaml에 test 추가됨
    split="test",  # ✅ "test" 데이터셋 사용
    imgsz=640,  # 이미지 크기
    batch=16,  # 배치 크기
    device="cuda"  # GPU 사용
)

# ✅ 평가 결과 출력 (mAP, Precision, Recall 등)
print(metrics)


Ultralytics 8.3.65 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/hyun/dev_ws/ROS_project/training/fire/data/mydata/train/labels... 1540 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1540/1540 [00:00<00:00, 2539.90it/s]


val: New cache created: /home/hyun/dev_ws/ROS_project/training/fire/data/mydata/train/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 97/97 [00:14<00:00,  6.69it/s]


                   all       1540       1540      0.997      0.997      0.995       0.94
                  fire        524        524      0.995      0.994      0.994      0.889
                 smoke       1016       1016          1          1      0.995      0.991
Speed: 0.3ms preprocess, 5.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7262e375ad20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.